In [ ]:
import socket
import threading

def client_thread(conn, addr, clients, usernames):
    username = conn.recv(1024).decode()
    usernames[conn] = username

    broadcast(f"{username} приєднався до чату", clients)

    while True:
        try:
            message = conn.recv(1024).decode()
            if not message:
                remove_client(conn, clients, usernames)
                break
            broadcast(f"{username}: {message}", clients)
        except:
            continue

def broadcast(message, clients):
    for client in clients:
        try:
            client.send(message.encode())
        except:
            continue

def remove_client(conn, clients, usernames):
    username = usernames[conn]
    clients.remove(conn)
    del usernames[conn]
    conn.close()
    broadcast(f"{username} вийшов з чату", clients)

def start_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(('localhost', 12346))
    server.listen()

    clients = []
    usernames = {}

    print("Сервер запущено. Очікування підключень...")

    while True:
        conn, addr = server.accept()
        clients.append(conn)

        thread = threading.Thread(target=client_thread, args=(conn, addr, clients, usernames))
        thread.start()

if __name__ == "__main__":
    start_server()

